In [32]:
import pandas as pd

# Notebook for aggregating NBA stats and salary data.

- Step 1: Merge players salary data with their performance.
- Step 2: Calculate relative salary as salary/salary_cap.
- Step 3: Calculate salary ratio as salary/total_team_salary
- Step 4: (TODO) Add search trend/popularity.

## Step 1: Merge players salary data with their performance.

Load salary and performance data for different seasons and combine them into a single dataframe.

In [33]:
salary_dfs = []

for year in range(2000, 2021):
    # Load the player's salary and performance respectively.
    partial_salary_df = pd.read_csv(f"../data/raw_data/player_salaries_{year + 1}.csv")
    partial_stats_df = pd.read_csv(f"../data/raw_data/player_stats_{year}.csv")
    # Format the players' names
    partial_salary_df[['Player']] = partial_salary_df[['Player']].applymap(lambda x: x.strip("*"))
    partial_stats_df[['Player']] = partial_stats_df[['Player']].applymap(lambda x: x.strip("*"))
    # Remove latent variables, all the removed variables can be recalculated in the future if needed.
    to_include = [i for i in partial_stats_df.columns if i[-1] != "%"]
    # Merge the player's income info and performance into a unified dataframe.
    full_player_data = pd.merge(partial_salary_df, partial_stats_df, on="Player", how="outer")
    full_player_data['Year'] = year
    full_player_data = full_player_data.dropna()
    salary_dfs.append(full_player_data)

# Merge data from different years into one dataframe
salary_df = pd.concat(salary_dfs)
del salary_dfs

salary_df[['Salary', 'Adjusted']] = salary_df[['Salary', 'Adjusted']].applymap(lambda x: int(x.strip('$').replace(',', '')))

salary_df

,Player,Salary,Adjusted,Pos,Age,Tm,G,GS,MP,FG,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year
0,Kevin Garnett,22400000,32441869,PF,23.0,MIN,81.0,81.0,40.0,9.4,...,2.8,9.0,11.8,5.0,1.5,1.6,3.3,2.5,22.9,2000
1,Shaquille O'Neal,21428572,31034952,C,27.0,LAL,79.0,79.0,40.0,12.1,...,4.3,9.4,13.6,3.8,0.5,3.0,2.8,3.2,29.7,2000
2,Alonzo Mourning,18754800,27162534,C,29.0,MIA,79.0,78.0,34.8,8.3,...,2.7,6.8,9.5,1.6,0.5,3.7,2.7,3.9,21.7,2000
3,Juwan Howard,18750000,27155582,PF,26.0,WAS,82.0,82.0,35.5,6.2,...,1.6,4.1,5.7,3.0,0.8,0.3,2.7,3.6,14.9,2000
4,Scottie Pippen,18083564,26190385,SF,34.0,POR,82.0,82.0,33.5,4.7,...,1.4,4.9,6.3,5.0,1.4,0.5,2.5,2.5,12.5,2000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
579,Mychal Mulder,449155,449155,SF,25.0,GSW,7.0,3.0,29.1,3.7,...,0.4,2.9,3.3,1.1,0.3,0.1,0.7,2.9,11.0,2020
582,Admiral Schofield,300000,300000,PF,22.0,WAS,33.0,2.0,11.2,1.1,...,0.2,1.2,1.4,0.5,0.2,0.1,0.2,1.5,3.0,2020
584,Jabari Parker,100000,100000,PF,24.0,TOT,38.0,23.0,24.1,5.8,...,1.6,4.0,5.6,1.8,1.2,0.4,1.7,2.6,14.0,2020
585,Jabari Parker,100000,100000,PF,24.0,ATL,32.0,23.0,26.2,6.2,...,1.7,4.3,6.0,1.8,1.3,0.5,1.9,2.9,15.0,2020


## Step 1.5: Clean duplicate player-rows to contain only total statistics.

## Step 2: Calculate relative salary compared to salary cap.

Load salary cap and format the monetary data.

In [34]:
salary_cap_df = pd.read_csv("../data/raw_data/salary_caps.csv")
salary_cap_df[['Salary Cap']] = salary_cap_df[['Salary Cap']].applymap(lambda x: int(x.strip('$').replace(',', '')))
salary_cap_df.head()

,Year,Salary Cap
0,2021,112414000
1,2020,109140000
2,2019,109140000
3,2018,101869000
4,2017,99093000


Merge the salary cap into the `salary_df` dataframe created and calculate the relative income.

In [35]:
salary_df = pd.merge(salary_df, salary_cap_df, on = "Year", how = "left")

## Step 3: Merge team's salary info to aggregated dataset.

Load team salaries.

In [36]:
team_salary_dfs = []
for year in range(2000, 2022):
    partial_team_salary_df = pd.read_csv(f"../data/raw_data/team_salaries_{year}.csv")
    partial_team_salary_df['Year'] = year
    team_salary_dfs.append(partial_team_salary_df)
team_salary_df = pd.concat(team_salary_dfs)
team_salary_df

,Team,Salary,Adjusted,Year
0,Portland,"$87,395,140","$130,685,637",2000
1,New York,"$74,007,738","$110,666,888",2000
2,Miami,"$73,472,329","$109,866,267",2000
3,Brooklyn,"$68,977,578","$103,145,079",2000
4,Washington,"$59,085,969","$88,353,738",2000
...,...,...,...,...
25,San Antonio,"$118,881,308","$118,881,308",2021
26,Charlotte,"$117,829,363","$117,829,363",2021
27,New Orleans,"$115,854,297","$115,854,297",2021
28,Memphis,"$115,017,735","$115,017,735",2021


The team(s) in which each player at in a given year in documented in abbreviates. We convert it to the full team name
with the fixed mapping defined here.

In [37]:
# Get mapping between full name and abbreviation
full_names = sorted(team_salary_df['Team'].unique().tolist())
abbr_names = sorted(salary_df['Tm'].unique().tolist())

# NBA team changes overtime, let's compensate for that.
indices = [4, 6, 20, 22, 23, 32, 34, 36]
names = ['Charlotte', 'Charlotte', 'Brooklyn', 'New Orleans', 'New Orleans', 'Oklahoma City', 'Total', 'Memphis']
for i, n in zip(indices, names):
    full_names.insert(i, n)

abbr_to_full_dict = dict(zip(abbr_names, full_names))
abbr_to_full_dict

{'ATL': 'Atlanta',
 'BOS': 'Boston',
 'BRK': 'Brooklyn',
 'CHA': 'Charlotte',
 'CHH': 'Charlotte',
 'CHI': 'Chicago',
 'CHO': 'Charlotte',
 'CLE': 'Cleveland',
 'DAL': 'Dallas',
 'DEN': 'Denver',
 'DET': 'Detroit',
 'GSW': 'Golden State',
 'HOU': 'Houston',
 'IND': 'Indiana',
 'LAC': 'LA Clippers',
 'LAL': 'LA Lakers',
 'MEM': 'Memphis',
 'MIA': 'Miami',
 'MIL': 'Milwaukee',
 'MIN': 'Minnesota',
 'NJN': 'Brooklyn',
 'NOH': 'New Orleans',
 'NOK': 'New Orleans',
 'NOP': 'New Orleans',
 'NYK': 'New York',
 'OKC': 'Oklahoma City',
 'ORL': 'Orlando',
 'PHI': 'Philadelphia',
 'PHO': 'Phoenix',
 'POR': 'Portland',
 'SAC': 'Sacramento',
 'SAS': 'San Antonio',
 'SEA': 'Oklahoma City',
 'TOR': 'Toronto',
 'TOT': 'Total',
 'UTA': 'Utah',
 'VAN': 'Memphis',
 'WAS': 'Washington'}

In [38]:
# Apply the mapping to the player stats.
salary_df["Team"] = salary_df[["Tm"]].applymap(lambda x: abbr_to_full_dict[x])
salary_df = salary_df.drop(columns=["Tm"])

Format the salary data.

In [39]:
team_salary_df[['Salary', "Adjusted"]] = team_salary_df[['Salary', "Adjusted"]].applymap(
    lambda x: int(x.strip('$').replace(',', '')))
team_salary_df

,Team,Salary,Adjusted,Year
0,Portland,87395140,130685637,2000
1,New York,74007738,110666888,2000
2,Miami,73472329,109866267,2000
3,Brooklyn,68977578,103145079,2000
4,Washington,59085969,88353738,2000
...,...,...,...,...
25,San Antonio,118881308,118881308,2021
26,Charlotte,117829363,117829363,2021
27,New Orleans,115854297,115854297,2021
28,Memphis,115017735,115017735,2021


In [40]:
salary_df = pd.merge(salary_df, team_salary_df, how="left", on=["Year", "Team"], suffixes=("_Player", "_Team"))
salary_df

,Player,Salary_Player,Adjusted_Player,Pos,Age,G,GS,MP,FG,FGA,...,STL,BLK,TOV,PF,PTS,Year,Salary Cap,Team,Salary_Team,Adjusted_Team
0,Kevin Garnett,22400000,32441869,PF,23.0,81.0,81.0,40.0,9.4,18.8,...,1.5,1.6,3.3,2.5,22.9,2000,35500000,Minnesota,47488500.0,71011552.0
1,Shaquille O'Neal,21428572,31034952,C,27.0,79.0,79.0,40.0,12.1,21.1,...,0.5,3.0,2.8,3.2,29.7,2000,35500000,LA Lakers,58968213.0,88177651.0
2,Alonzo Mourning,18754800,27162534,C,29.0,79.0,78.0,34.8,8.3,15.0,...,0.5,3.7,2.7,3.9,21.7,2000,35500000,Miami,73472329.0,109866267.0
3,Juwan Howard,18750000,27155582,PF,26.0,82.0,82.0,35.5,6.2,13.5,...,0.8,0.3,2.7,3.6,14.9,2000,35500000,Washington,59085969.0,88353738.0
4,Scottie Pippen,18083564,26190385,SF,34.0,82.0,82.0,33.5,4.7,10.5,...,1.4,0.5,2.5,2.5,12.5,2000,35500000,Portland,87395140.0,130685637.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6725,Mychal Mulder,449155,449155,SF,25.0,7.0,3.0,29.1,3.7,9.6,...,0.3,0.1,0.7,2.9,11.0,2020,109140000,Golden State,171105334.0,171105334.0
6726,Admiral Schofield,300000,300000,PF,22.0,33.0,2.0,11.2,1.1,2.8,...,0.2,0.1,0.2,1.5,3.0,2020,109140000,Washington,131294012.0,131294012.0
6727,Jabari Parker,100000,100000,PF,24.0,38.0,23.0,24.1,5.8,11.3,...,1.2,0.4,1.7,2.6,14.0,2020,109140000,Total,NaN,NaN
6728,Jabari Parker,100000,100000,PF,24.0,32.0,23.0,26.2,6.2,12.3,...,1.3,0.5,1.9,2.9,15.0,2020,109140000,Atlanta,118804016.0,118804016.0


## Step 4: Merge team statistics to aggregated data.

In [41]:
from config import mapping

team_stats_dfs = []

for year in range(2000, 2022):
    # Load the player's salary and performance respectively.
    partial_stats_df = pd.read_csv(f"../data/raw_data/team_stats_{year}.csv")
    # Format the players' names
    partial_stats_df[['Team']] = partial_stats_df[['Team']].applymap(lambda x: x.strip("*"))
    partial_stats_df["Year"] = year
    # Remove latent variables, all the removed variables can be recalculated in the future if needed.
    to_include = [i for i in partial_stats_df.columns if i[-1] != "%"]
    # Merge the player's income info and performance into a unified dataframe.
    team_stats_dfs.append(partial_stats_df[to_include])

team_stats_df = pd.concat(team_stats_dfs)
team_stats_df[['Team']] = team_stats_df[['Team']].applymap(lambda x: mapping[x])
salary_df = pd.merge(salary_df, team_stats_df, how = "left", on = ['Team', 'Year'], suffixes=("_Player", "_Team"))
salary_df

,Player,Salary_Player,Adjusted_Player,Pos,Age,G_Player,GS,MP_Player,FG_Player,FGA_Player,...,FTA_Team,ORB_Team,DRB_Team,TRB_Team,AST_Team,STL_Team,BLK_Team,TOV_Team,PF_Team,PTS_Team
0,Kevin Garnett,22400000,32441869,PF,23.0,81.0,81.0,40.0,9.4,18.8,...,21.6,12.4,30.1,42.5,26.9,7.6,5.4,13.9,23.3,98.5
1,Shaquille O'Neal,21428572,31034952,C,27.0,79.0,79.0,40.0,12.1,21.1,...,28.9,13.6,33.4,47.0,23.4,7.5,6.5,13.9,22.5,100.8
2,Alonzo Mourning,18754800,27162534,C,29.0,79.0,78.0,34.8,8.3,15.0,...,22.3,11.2,31.9,43.2,23.5,7.1,6.4,15.0,23.7,94.4
3,Juwan Howard,18750000,27155582,PF,26.0,82.0,82.0,35.5,6.2,13.5,...,25.7,13.0,29.7,42.7,21.6,7.2,4.7,16.1,26.2,96.6
4,Scottie Pippen,18083564,26190385,SF,34.0,82.0,82.0,33.5,4.7,10.5,...,24.7,11.8,31.2,43.0,23.5,7.7,4.8,15.2,22.7,97.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6942,Mychal Mulder,449155,449155,SF,25.0,7.0,3.0,29.1,3.7,9.6,...,23.2,10.0,32.9,42.8,25.6,8.2,4.6,14.9,20.1,106.3
6943,Admiral Schofield,300000,300000,PF,22.0,33.0,2.0,11.2,1.1,2.8,...,24.6,10.2,31.9,42.0,25.0,8.0,4.3,14.2,22.7,114.4
6944,Jabari Parker,100000,100000,PF,24.0,38.0,23.0,24.1,5.8,11.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6945,Jabari Parker,100000,100000,PF,24.0,32.0,23.0,26.2,6.2,12.3,...,23.4,9.9,33.4,43.3,24.0,7.8,5.1,16.2,23.1,111.8


## Step 5: Add search trend/popularity

In [42]:
queries = []
cache = {}

In [43]:
google_trends = pd.read_csv("../data/clean_data/trends_cleaned.csv")
google_trends = google_trends.T.reset_index()
google_trends = google_trends.rename(columns = google_trends.iloc[0])
google_trends = google_trends.drop(0)
google_trends = google_trends.rename(columns ={"date": "Player"})
google_trends = google_trends.set_index("Player")
google_trends

,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
Player,,,,,,,,,,,,,,,,,
Kevin Garnett,388,185,175,289,344,268,301,225,360,398,220,289,219,131,115,110,205
Shaquille O'Neal,612,418,476,475,329,277,248,272,194,151,143,194,252,218,206,225,236
Alonzo Mourning,330,542,368,256,253,315,271,321,374,362,398,228,246,231,217,237,246
Juwan Howard,94,49,51,66,50,47,150,207,264,248,101,67,51,33,40,197,68
Hakeem Olajuwon,149,265,141,203,203,301,300,334,475,565,444,492,397,384,386,345,498
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Neemias Queta,0,0,0,0,0,0,0,0,0,0,0,0,0,0,12,630,223
Joe Wieskamp,0,0,0,0,0,0,0,0,0,0,0,80,66,113,263,348,333
Justin Champagnie,0,0,0,0,0,0,58,0,0,0,0,0,0,0,73,298,299


In [44]:
from tqdm import tqdm

trends = []
years = list(google_trends.columns)

for i in tqdm(range(len(queries), len(salary_df))):
    row = salary_df.iloc[i]
    player = row["Player"]
    year = row["Year"]
    if player in google_trends.index:
        if int(year) in years:
            trends.append(google_trends.loc[player][year])
        else:
            trends.append(None)
    else:
        trends.append(None)

100%|██████████| 6947/6947 [00:01<00:00, 4406.49it/s]


## Step 6: Save the dataset to local.

In [45]:
salary_df["Google Search Trend"] = trends
salary_df

,Player,Salary_Player,Adjusted_Player,Pos,Age,G_Player,GS,MP_Player,FG_Player,FGA_Player,...,ORB_Team,DRB_Team,TRB_Team,AST_Team,STL_Team,BLK_Team,TOV_Team,PF_Team,PTS_Team,Google Search Trend
0,Kevin Garnett,22400000,32441869,PF,23.0,81.0,81.0,40.0,9.4,18.8,...,12.4,30.1,42.5,26.9,7.6,5.4,13.9,23.3,98.5,NaN
1,Shaquille O'Neal,21428572,31034952,C,27.0,79.0,79.0,40.0,12.1,21.1,...,13.6,33.4,47.0,23.4,7.5,6.5,13.9,22.5,100.8,NaN
2,Alonzo Mourning,18754800,27162534,C,29.0,79.0,78.0,34.8,8.3,15.0,...,11.2,31.9,43.2,23.5,7.1,6.4,15.0,23.7,94.4,NaN
3,Juwan Howard,18750000,27155582,PF,26.0,82.0,82.0,35.5,6.2,13.5,...,13.0,29.7,42.7,21.6,7.2,4.7,16.1,26.2,96.6,NaN
4,Scottie Pippen,18083564,26190385,SF,34.0,82.0,82.0,33.5,4.7,10.5,...,11.8,31.2,43.0,23.5,7.7,4.8,15.2,22.7,97.5,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6942,Mychal Mulder,449155,449155,SF,25.0,7.0,3.0,29.1,3.7,9.6,...,10.0,32.9,42.8,25.6,8.2,4.6,14.9,20.1,106.3,191.0
6943,Admiral Schofield,300000,300000,PF,22.0,33.0,2.0,11.2,1.1,2.8,...,10.2,31.9,42.0,25.0,8.0,4.3,14.2,22.7,114.4,70.0
6944,Jabari Parker,100000,100000,PF,24.0,38.0,23.0,24.1,5.8,11.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.0
6945,Jabari Parker,100000,100000,PF,24.0,32.0,23.0,26.2,6.2,12.3,...,9.9,33.4,43.3,24.0,7.8,5.1,16.2,23.1,111.8,40.0


In [46]:
import numpy as np

salary_df = salary_df.replace(np.nan, '', regex = True)
salary_df.to_csv("../data/clean_data/player_salary.csv", index=False)

In [47]:
salary_df.columns

Index(['Player', 'Salary_Player', 'Adjusted_Player', 'Pos', 'Age', 'G_Player',
       'GS', 'MP_Player', 'FG_Player', 'FGA_Player', 'FG%', '3P_Player',
       '3PA_Player', '3P%', '2P_Player', '2PA_Player', '2P%', 'eFG%',
       'FT_Player', 'FTA_Player', 'FT%', 'ORB_Player', 'DRB_Player',
       'TRB_Player', 'AST_Player', 'STL_Player', 'BLK_Player', 'TOV_Player',
       'PF_Player', 'PTS_Player', 'Year', 'Salary Cap', 'Team', 'Salary_Team',
       'Adjusted_Team', 'G_Team', 'MP_Team', 'FG_Team', 'FGA_Team', '3P_Team',
       '3PA_Team', '2P_Team', '2PA_Team', 'FT_Team', 'FTA_Team', 'ORB_Team',
       'DRB_Team', 'TRB_Team', 'AST_Team', 'STL_Team', 'BLK_Team', 'TOV_Team',
       'PF_Team', 'PTS_Team', 'Google Search Trend'],
      dtype='object')